Progetto: **Classificazione di testi descrittivi per destinazione d'uso, inerenti ai beni sottoposti ad aste giudiziarie italiane.**

Studente: **Alessandro Monolo** | *10439147*

Relatore: Marco Brambilla

Referente aziendale: Simone Redaelli

Master: Data Science & Artificial Intelligence

Università: Politecnico di Milano

<hr style="border:1px solid black">

# Filtro file JSON estratto da MongoDB con le colonne inerenti al progetto

- **1.** **Filtro** il data frame estratto da MongoDB con tutte le colonne che mi servono;


- **2.** **Analizzo** il data frame;


- **3.** **Salvo** il data frame filtrato e corretto in un fomato **CSV**.

<hr style="border:1px solid black">

**Importo le librerie che mi servono:**

In [1]:
import pandas as pd

#### Set pandas options:

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 1000
pd.options.mode.chained_assignment = None 

**Importo file JSON**

In [3]:
df_json = pd.read_json('D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\0_Dataset_MongoDB\\10.08.2023\\Descrizioni beni DD2.json')

### 1 - Seleziono le colonne con le quali mi serve lavorare

In [4]:
column_list = ["DestinazioneUso",
               "CategoriaCatastale",
               "Provincia",
               "Comune",
               "Tribunale",
               "NumLotto",
               "Superficie",
               "NumeroVani",
               "TipoCatasto",
               "Descrizione"]

**Filtro il Data Frame Pandas**

In [5]:
df = df_json.loc[:, column_list]

### 2 - Controllo il df estratto appena filtrato

In [6]:
df.shape

(251510, 10)

- Il Data Frame è composto da **10** colonne e **251.510** righe;

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251510 entries, 0 to 251509
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   DestinazioneUso     251510 non-null  object
 1   CategoriaCatastale  251510 non-null  object
 2   Provincia           251510 non-null  object
 3   Comune              251510 non-null  object
 4   Tribunale           251510 non-null  object
 5   NumLotto            251510 non-null  object
 6   Superficie          190981 non-null  object
 7   NumeroVani          108586 non-null  object
 8   TipoCatasto         251510 non-null  object
 9   Descrizione         251472 non-null  object
dtypes: object(10)
memory usage: 19.2+ MB


- Attualmente **nessuna** colonna è espressa in valori **numerici**;
- Tutte le colonne sono quindi **Object** Format.

In [8]:
print("Somma delle Null Values per singola colonna del data frame:")
print(df.isnull().sum())
print("\n")
print("Null Values espresse in termini percentuali per singola colonna del data frame:")
print(df.isnull().sum()*100 / len(df))

Somma delle Null Values per singola colonna del data frame:
DestinazioneUso            0
CategoriaCatastale         0
Provincia                  0
Comune                     0
Tribunale                  0
NumLotto                   0
Superficie             60529
NumeroVani            142924
TipoCatasto                0
Descrizione               38
dtype: int64


Null Values espresse in termini percentuali per singola colonna del data frame:
DestinazioneUso        0.000000
CategoriaCatastale     0.000000
Provincia              0.000000
Comune                 0.000000
Tribunale              0.000000
NumLotto               0.000000
Superficie            24.066240
NumeroVani            56.826369
TipoCatasto            0.000000
Descrizione            0.015109
dtype: float64


- La maggior parte delle colonne non presenta Null Values al loro interno;
- Le uniche sono le colonne "**Superficie**" e "**NumeroVani**".
    - Questa presenza massiccia di Null Values può essere inerente al fatto che all'interno del data frame i suddetti campi vengono/possono essere mappati solo nel caso in cui non si tratti di terreni (ovvero beni con destinazione d'uso equivalente a LAND);
- Il campo "**Descrizione**" è vuoto in 38 casi;
    - Devo quindi rimuovere tutte quelle righe dove in questo campo non esistono stringhe descrittive del bene, prima di esportare il data frame appena filtrato.

In [9]:
df.describe()

,DestinazioneUso,CategoriaCatastale,Provincia,Comune,Tribunale,NumLotto,Superficie,NumeroVani,TipoCatasto,Descrizione
count,251510,251510,251510,251510,251510,251510,190981,108586,251510,251472
unique,8,53,109,6379,190,3159,5369,240,2,154601
top,RESIDENTIAL,A2,RM,ROMA,BERGAMO,1,0,5,F,Appartamento al piano primo.
freq,175185,50771,20882,11376,14790,151590,115300,10603,241555,815


- Ci sono **8** classi nel campo **destinazione d'uso**;
    - La più frequente è la classe "**RESIDENTIAL**", presente **175.185** volte, con categoria catastale **A2**;
- Il comune più citato è il comune di ROMA;
- Mentre il tribunale di assegnazione più frequente nel data frame è quello di **Bergamo**;
- Da notare inoltre che la Mq minima è pari a zero, quindi un ipotetico errore da indagare successivamente.

**Droppo tutte le righe dove in entrambi i campi Descrizione e DescrizioneLotto i valori sono assenti**

In [10]:
df_filtrato = df.dropna(subset=['Descrizione'], how='all')

**Stampo ora il totale delle righe droppate**

In [11]:
print("Numero righe rimosse: ", df.shape[0] - df_filtrato.shape[0])

Numero righe rimosse:  38


- Sono stati eliminati **38 beni** dove nel campo Descrizione era effettivamente presente una descrizione;

### Elimino tutti i beni che non ha una Destinazione D'Uso, ovvero tutti quei beni che sono identificati come "OTHER" nella Target Variable.

In [12]:
df_filtrato = df_filtrato[df_filtrato["DestinazioneUso"] != "OTHER"]

#### Prima di salvare il nuovo data frame filtrato, sostituisco la stringa "NA" con "NAP" per la provincia di Napoli, al fine di evitare che durante l'import nei prossimi step possa essere letta come una Null Values

In [13]:
# Sostituisco la stringa NA con NAP per evitare null values:
df_filtrato.loc[df_filtrato["Provincia"] == "NA", "Provincia"] = "NAP"

### 3 - Procedo a salvare il data frame filtrato in un file formato CSV: 

In [14]:
df_filtrato_csv = df_filtrato.to_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\1_Dataset_Filtrato\\Descrizioni_Filtrate_10.08.2023.csv",
                                     index=False)